In [1]:
import pickle
import pandas as pd
from scipy import sparse

In [5]:
!head -n 2 sample_processed.csv


text,industry,cluster
benefits board benefits board homejoin planning tab navigate menu items view account online member account portal provides access information state art security encryption join plan visit membership overview page learn minister retirement plan benefits board currently offering matching contributions new participants church loan fund low rates refinancing know signup subscribe nowcongrats subscribed knowconnect toll free local fax mail info benefitsboard com deposit funds make contribution deposits using secure electronic payment system enroll first time user know april final instructions passed away today would family know access important accounts files insurance policies even social media accounts current pandemic contributed hundreds thousands deaths people would likely lived much longer lives many unexpected deaths read church god benefits board cleveland privacy policy,marketing and advertising,0


In [6]:
data = pd.read_csv('sample_processed.csv')
data.head(10)

,text,industry,cluster
0,benefits board benefits board homejoin plannin...,marketing and advertising,0
1,home columbia landcare environmental stewardsh...,environmental services,4
2,region inc worldwide revenue accelerating worl...,management consulting,3
3,home team careers services civil design case s...,civil engineering,4
4,home friedman associates friedman associates c...,accounting,3
5,big rapids real estate search houses homes sal...,real estate,3
6,garcia law firm garcia law firm home attorney ...,legal services,3
7,guardian athletics kato collar invest online s...,sports,0
8,miracle doors america premier door service com...,utilities,4
9,valerion biotechnology company focused develop...,biotechnology,0


In [7]:
with open('vectors.pkl', 'rb') as f:
    vectors = pickle.load(f)

vectors.shape

(215160, 1611989)

In [8]:
def cosine(x:sparse.spmatrix,y: sparse.spmatrix):
    return x.multiply(y).sum() / (x.dot(x.T).sqrt().sum() * y.dot(y.T).sqrt().sum())

cosine(vectors[5], vectors[7])

0.016355522371438964

In [ ]:
_df = data[data['cluster'] == 0]

pairs = (
    (i1, i2) 
    for i1, x1 in _df.iterrows() 
    for i2, x2 in _df.iterrows() 
    if i1 < i2 and x1.industry != x2.industry
)

for t in range(2):
    print('---')
    i,j = next(pairs)
    print((i,j,cosine(vectors[i], vectors[j])))

In [9]:
import threading

# _df = data[data['cluster'] == 0]

# pairs = (
#     (i1, i2) 
#     for i1, x1 in _df.iterrows() 
#     for i2, x2 in _df.iterrows() 
#     if i1 < i2 and x1.industry != x2.industry
# )

# for t in range(2):
#     print('---')
#     i,j = next(pairs)
#     print((i,j,cosine(vectors[i], vectors[j])))

class ClusterPairwiseSimilarityThread(threading.Thread):
    def __init__(self, df, cluster_id, vectors, output_file) -> None:
        threading.Thread.__init__(self)
        assert 'cluster' in df.columns
        assert 'industry' in df.columns
        self.df = df
        self.vectors = vectors
        self.cluster_id = cluster_id
        self.output_file = output_file
    
    def __get_pairs(self):
        _df = self.df[self.df.cluster == self.cluster_id]
        return (
            (i1, i2) 
            for i1, x1 in _df.iterrows() 
            for i2, x2 in _df.iterrows() 
            if i1 < i2 and x1.industry != x2.industry
        )

    def run(self):
        print('Calculating pairwise cosine similarity for cluster ' + str(self.cluster_id))
        with open(self.output_file, 'w') as out:
            for i,j in self.__get_pairs():
                out.write(f'{i},{j},{cosine(self.vectors[i], self.vectors[j])}\n')
        print('Finished cluster ' + str(self.cluster_id))
        

In [10]:
threads = []

In [11]:
t = ClusterPairwiseSimilarityThread(data, 0, vectors, 'cluster_0_similarities.txt')
t.start()
threads.append(t)

Calculating pairwise cosine similarity for cluster 0


In [13]:
for c in data.cluster.unique():
    if c != 0: # DELETE ME
        t = ClusterPairwiseSimilarityThread(data, c, vectors, f'cluster_{c}_similarities.txt')
        t.start()
        threads.append(t)

Calculating pairwise cosine similarity for cluster 4
Calculating pairwise cosine similarity for cluster 3
Calculating pairwise cosine similarity for cluster 2
Calculating pairwise cosine similarity for cluster 1
Calculating pairwise cosine similarity for cluster 5


In [6]:
industries = data.industry.unique()
clusters = data.cluster.unique()

distance = {}

for i in industries:
    for c in clusters:
        print(i,c)
        left = data[(data.industry == i) & (data.cluster == c)]
        if left.shape[0] > 0:
            right = data[(data.industry != i) & (data.cluster == c)]
            assert right.shape[0] > 0

            for 
            # dist = pairwise_distances(vectors[left.index], vectors[right.index], metric='cosine')
        break
    break

marketing and advertising 3


In [7]:
left

,Unnamed: 0,text,industry,cluster
0,0,benefits board benefits board homejoin plannin...,marketing and advertising,3
207,207,green card wisconsin newest discount golf card...,marketing and advertising,3
214,214,home cubed site homeabout ushow worksrequest d...,marketing and advertising,3
317,317,gauger associates case studies capabilities cl...,marketing and advertising,3
365,365,bloc influencers agency bloc services influenc...,marketing and advertising,3
...,...,...,...,...
1694246,1694246,shipley brown media group home media solutions...,marketing and advertising,3
1694309,1694309,box home courses tob teams testimonials jobs b...,marketing and advertising,3
1694372,1694372,fms expert management advisory services field ...,marketing and advertising,3
1694482,1694482,welcome professional manufacturers pmr toggle ...,marketing and advertising,3


In [8]:
right

,Unnamed: 0,text,industry,cluster
5,5,big rapids real estate search houses homes sal...,real estate,3
7,7,advfn free stock cryptocurrency prices charts ...,internet,3
8,8,guardian athletics kato collar invest online s...,sports,3
10,10,valerion biotechnology company focused develop...,biotechnology,3
11,11,test page latticesystems com html body div spa...,e-learning,3
...,...,...,...,...
1694577,1694577,warehouse management systems software wms syst...,computer software,3
1694581,1694581,home byron bethany irrigation district menu ho...,government administration,3
1694582,1694582,official online store ewing athletics ewing at...,apparel & fashion,3
1694583,1694583,staffing undercover productions las vegas unde...,entertainment,3


In [14]:
from itertools import combinations

In [1]:
to_compute = {}

for i in left.index:
    for j in right.index:
        to_compute[(i,j)] = 0

to_compute

NameError: name 'left' is not defined

In [ ]:
to_compute

In [ ]:
cosine_values = pairwise_distances(X=tfidf_industry, Y=tfidf_other, metric='cosine')
cosine_values = scipy.sparse.csr_matrix(cosine_values)
indices = cosine_values.argmax(axis=1)
for i, index in enumerate(current_industry_indices):
    related_of[index] = current_other_indices[indices[i,0]]
print(cosine_values)